In [ ]:
import sys, os
import matplotlib
import cartopy
import xarray as xr

In [ ]:
%matplotlib inline

In [ ]:
from opendrift.models.oceandrift import OceanDrift
from opendrift.readers import reader_ROMS_native

import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

np.random.seed(983214)

In [ ]:
fjordos_geo = xr.open_dataset("https://thredds.met.no/thredds/dodsC/fjordos/geogrid_tmp/test_polstere_1h_0007.nc")
fjordos_geo

In [ ]:
fjordos = xr.open_dataset("/lustre/storeB/project/fou/hi/FjordOs/fjordos2_hindcast/v6/exp1/ocean_his_0007.nc4")
fjordos

In [ ]:
o = OceanDrift(loglevel=50)

print(o)

In [ ]:
#norkyst_model_input_files = [
#                "https://thredds.met.no/thredds/dodsC/fou-hi/norkyst800m-1h/NorKyst-800m_ZDEPTHS_his.an.2019072100.nc"
#            ]

fjordos_model_input_files = [
                "/lustre/storeB/project/fou/hi/FjordOs/fjordos2_hindcast/v6/exp1/ocean_his_0007.nc4"
            ]
#                "https://thredds.met.no/thredds/dodsC/fjordos/fjordos2_hindcast/v6/exp1/ocean_his_0007.nc4"
#            ]

reader = reader_ROMS_native.Reader(fjordos_model_input_files)
print(reader)

In [ ]:
reader.plot()

In [ ]:
o.add_reader([reader])

In [ ]:
print("First available timestep in reader: {}".format(reader.start_time))
print("Last available timestep in reader: {}".format(reader.end_time))

sim_start_time_index = 5*24
sim_end_time_index = 5*24 + 6

sim_start_time = reader.times[sim_start_time_index]
sim_end_time = reader.times[sim_end_time_index]

print("First timestep of simulation: {}".format(sim_start_time))
print("Last timestep of simulation: {}".format(sim_end_time))

In [ ]:
# random seeding in the middle of the inner Oslofjord
#o.seed_elements(lon=10.55, lat=59.80, number=10, radius=500,
#                    time=sim_start_time)


seed_points_lats = [59.78610610961914,
                  59.700775146484375,
                  59.74568176269531,
                  59.8759651184082,
                  59.822044372558594,
                  59.81306076049805]

seed_points_lons = [10.55042552947998,
                  10.58590030670166,
                  10.53268814086914,
                  10.656847953796387,
                  10.710057258605957,
                  10.5149507522583]

n_seeds = len(seed_points_lats)

o.seed_elements(lat=seed_points_lats, lon=seed_points_lons, 
                time=sim_start_time, 
                wind_drift_factor=np.zeros_like(seed_points_lats))

In [ ]:
print(o)

In [ ]:
print(o.list_configspec())

In [ ]:
#o.run(end_time=sim_end_time, time_step=60, time_step_output=60, outfile='opendrift.nc') # sim & dump every minute
o.run(end_time=sim_end_time, time_step=600, time_step_output=600, outfile='opendrift.nc') # sim & dump every 10 minutes

In [ ]:
o.plot()

In [ ]:
print(o)

In [ ]:
print(len(o.get_lonlats()))
print(o.get_lonlats()[0].shape) # (n_drifters, n_timesteps)

print(o.get_lonlats()[0][0]) # lons for drifter 0
print(o.get_lonlats()[1][0]) # lats for drifter 0

print(o.get_time_array()[0]) # list of times (datetime)

print(reader.lonlat2xy(o.get_lonlats()[0][0], o.get_lonlats()[1][0]))

In [ ]:
time = o.get_time_array()[0]

lon = o.get_lonlats()[0]
lat = o.get_lonlats()[1]

x = []
y = []
for i in range(n_seeds):
    x_i, y_i = reader.lonlat2xy(o.get_lonlats()[0][i], o.get_lonlats()[1][i])
    x.append(x_i.tolist())
    y.append(y_i.tolist())

opendrift_results = {"time": o.get_time_array()[0],
                    "lon": lon,
                    "lat": lat,
                    "x": x,
                    "y": y}

print(opendrift_results)

In [ ]:
import pickle

with open("fjordos_opendrift_trajectories.pickle", "wb") as f:
    pickle.dump(opendrift_results, f, protocol=4)

In [ ]:
with open("fjordos_opendrift_trajectories.pickle", "rb") as f:
    opendrift_results = pickle.load(f)